In [1]:
import cv2
import numpy as np
from keras.preprocessing import image as im
from keras import backend as k
import tensorflow as tf
import pytesseract as tesseract
tf.reset_default_graph()
from keras.models import load_model
model=load_model('finalbestmodel.hdf5')

C:\Users\Praneet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Get tags from non-filled image
def traverseCroppedSections():
    labelList = []
    for x, y, w, h in coordinates:
        croppedSection = cropImage(x, y, w, h, nonfilledtesseract)
        label = tesseract.image_to_string(croppedSection)
        labelList.append(label)
    print(labelList)
    # push these labels to database along with the coordinates and index
    
# Cropping the image
def cropImage(x, y, w, h, image):
    # Crop image here
    croppedImage = image[y:y + h,x:x + w]
    return croppedImage

# function to match template
def templateMatching(wholeimage, template):
    result = cv2.matchTemplate(wholeimage, template, cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    return max_loc[0], max_loc[1]

def makeSquare(not_square):
    # Adds black pixels as padding
    
    BLACK = [0, 0, 0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square, (2 * width, 2 * height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        if (height > width):
            pad = (height - width)/2
            pad = int(pad)
            #doublesize = int(doublesize)
            doublesize_square = cv2.copyMakeBorder(doublesize, 0, 0, pad,\
                                                  pad, cv2.BORDER_CONSTANT, value = BLACK)
        else:
            pad = (width - height)/2
            pad = int(pad)
            doublesize_square = cv2.copyMakeBorder(doublesize, pad, pad, 0, 0,\
                                                  cv2.BORDER_CONSTANT, value = BLACK)
    doublesize_square_dim = doublesize_square.shape
    return doublesize_square

def resize_to_pixel(dimensions, image):
    
    buffer_pix = 4
    dimensions = dimensions - buffer_pix
    squared = image
    r = float(dimensions)/squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    widht_r = img_dim2[1]
    BLACK = [0, 0, 0]
    if (height_r > widht_r):
        resized = cv2.copyMakeBorder(resized, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value = BLACK)
    if (height_r < widht_r):
        resized = cv2.copyMakeBorder(resized, 1, 0, 0, 0, cv2.BORDER_CONSTANT, value = BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized, p, p, p, p, cv2.BORDER_CONSTANT, value = BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    return ReSizedImg


def prediction(char_image):
    squared = makeSquare(char_image)
    size28 = resize_to_pixel(28, squared)
    cv2.imshow('char resized', size28)
    cv2.waitKey(0)
    cv2.destroyAllWindows()     
    predict_img = im.img_to_array(size28)
    predict_img = np.expand_dims(predict_img, axis = 0)
    predictedarray = model.predict(predict_img)
    index = calculateClass(predictedarray[0])
    
    predicted_class = label_dictionary[index]
    return predicted_class

def calculateClass(predictedarray):
    predictedclass = 0
    predictedclassindex = 0
    index = 0
    for classprediction in predictedarray:
        if classprediction > predictedclass:
            predictedclass = classprediction
            predictedclassindex = index
        index = index + 1
    return predictedclassindex

def histogramSegmentation(img):
    #int(img.shape[1] * (1/5))
    print(img.shape[1])
    THRESHOLD = int(img.shape[1] * (1/7.5))
    print(THRESHOLD)
    histogramResults = []
    
    col_histo_width = img.shape[1]
    col_histo_height = img.shape[0]
    col_histogram = np.zeros((col_histo_height, col_histo_width), np.uint8)

    # Initializing horizontal histogram
    row_histo_height = img.shape[0]
    row_histo_width = img.shape[1]
    row_histogram = np.zeros((row_histo_height, row_histo_width), np.uint8)

    colsums = []
    startflag = False
    endflag = True
    for col in range(col_histo_width):
        running_sum = sum(img[:, col]) // 255
        if startflag == False and running_sum >= THRESHOLD:
            startindex = col
            if (startindex < 0):
                startindex = 0
            startflag = True
            endflag = False
        if endflag == False and running_sum <= THRESHOLD:
            endindex = col
            if endindex > col_histo_width:
                endindex = col_histo_width
            endflag = True
            startflag = False
            if startindex != endindex:
                histogramResults.append((startindex, endindex))
                print((startindex, endindex))
        colsums.append(running_sum)
    return histogramResults


# cropping and setdifferencing function
def getNonfilledCroppedSections(coordinates):
    i = 0
    for x, y, w, h in coordinates:
        
        tag = tags[i]
        i = i + 1
        
        # cropping non filled image
        nonfilledcroppedsection = cropImage(x, y, w, h, nonfilledimage)
        cv2.imshow('nonfilledcroppedsection', nonfilledcroppedsection)
        cv2.waitKey(0)
        
        #dilating 
        nonfilledcroppedsectiondilated = cropImage(x, y, w, h, nonfilledimagedilated)
        
        crop_h = nonfilledcroppedsection.shape[0]
        crop_w = nonfilledcroppedsection.shape[1]
        
        buffer_x = x - 75
        buffer_y = y - 75
        buffer_w = w + 2*75
        buffer_h = h + 2*75
        
        if ( buffer_x < 0 ):
            buffer_x = 0
        if ( buffer_y < 0 ):
            buffer_y = 0
        if ( buffer_x + buffer_w > filledimage.shape[0] ):
            buffer_w = filledimage.shape[0]
        if ( buffer_y + buffer_h > filledimage.shape[1] ):
            buffer_h = filledimage.shape[1]
            
        filledcroppedsectionbuffer = cropImage(buffer_x, buffer_y, buffer_w, buffer_h, filledimagedilated)
        cv2.imshow('filledcroppedsectionbuffer', filledcroppedsectionbuffer)
        cv2.waitKey(0)
        
        # To use for set difference
        filledcroppedsectionbuffernondilated = cropImage(buffer_x, buffer_y, buffer_w, buffer_h, filledimage)
        
        #crop_x, crop_y = templateMatching(filledcroppedsectionbuffer, nonfilledcroppedsection)
        crop_x, crop_y = templateMatching(filledcroppedsectionbuffernondilated, nonfilledcroppedsection)
        
        # cropping the matched template from filled image
        filledcroppedsection = cropImage(buffer_x + crop_x, buffer_y + crop_y, crop_w, crop_h, filledimage)
        cv2.imshow('filledcroppedsection', filledcroppedsection)
        cv2.waitKey(0)
        
        # subtracting nonfilled image from filled image
        setDifference = np.subtract(filledcroppedsection, nonfilledcroppedsection)
        cv2.imshow('setDifference', setDifference)
        cv2.waitKey(0)
        
        setDifference = cv2.medianBlur(setDifference, 3)
        #setDifference = cv2.erode(setDifference, np.ones((3, 3), np.uint8), iterations=1)
        cv2.imshow('setDifferenceeroded', setDifference)
        cv2.waitKey(0)
        
        converted_to_text = ocr(setDifference)
        print(tag + ' : ' + converted_to_text)
        
        
"""def ocr(image):
    
    # detecting edges in the image
    image_edges = cv2.Canny(image, 30, 150)
    
    # dilating image to detect individual lines
    kernel_line = np.ones((10, 80), np.uint8)
    dilated_line = cv2.dilate(image_edges, kernel_line, iterations=1)
    
    # finding contours of the line
    im2, ctrs_line, hier = cv2.findContours(dilated_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs_line = sorted(ctrs_line, key=lambda ctr: cv2.boundingRect(ctr)[1])
    
    # list of words
    word_list = []
    
    for i, ctr_line in enumerate(sorted_ctrs_line):
        # getting coordinates of the line contour
        line_x, line_y, line_w, line_h = cv2.boundingRect(ctr_line)
        
        # if condition for removing unnecessary contours
        if line_w * line_h < 10000:
            continue
        
        line = cropImage(line_x, line_y, line_w, line_h, image)
        cv2.imshow('line', line)
        cv2.waitKey(0)
        
        # detecting edges in the image
        line_edges = cv2.Canny(line, 30, 150)
        
        # dilating line to detect individual words
        kernel_word = np.ones((10, 60), np.uint8)
        dilated_word = cv2.dilate(line_edges, kernel_word, iterations=1)
        
        # finding contours of the word
        im2, ctrs_word, hier = cv2.findContours(dilated_word.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        sorted_ctrs_word = sorted(ctrs_word, key=lambda ctr: cv2.boundingRect(ctr)[0])
        
        for i, ctr_word in enumerate(sorted_ctrs_word):
            # getting coordinates of the word contour
            word_x, word_y, word_w, word_h = cv2.boundingRect(ctr_word)
            
            # if condition for removing unnecessary contours
            if word_w * word_h < 2500:
                continue
            
            #print("Word " + str(word_x), str(word_y), str(word_w), str(word_h))
            
            word = cropImage(word_x, word_y, word_w, word_h, line)
            cv2.imshow('word', word)
            cv2.waitKey(0)
            
            word = cv2.erode(word, np.ones((1, 1), np.uint8), iterations=1)
            
            # detecting edges in the image
            word_edges = cv2.Canny(word, 30, 150)
            
            # dilating word to detect individual characters
            kernel_char = np.ones((15, 3), np.uint8)
            dilated_char = cv2.dilate(word_edges, kernel_char, iterations=1)
            cv2.imshow('dilated_char', dilated_char)
            cv2.waitKey(0)
            
            # finding contours of the characters
            im2, ctrs_char, hier = cv2.findContours(dilated_char.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            sorted_ctrs_char = sorted(ctrs_char, key=lambda ctr: cv2.boundingRect(ctr)[0])
            
            # list of words
            character_list = []
            
            for i, ctr_char in enumerate(sorted_ctrs_char):
                # getting coordinates of the character contour
                char_x, char_y, char_w, char_h = cv2.boundingRect(ctr_char)
                
                # if condition for removing unnecessary contours
                if char_w * char_h < 180:
                    continue
                    
                #print("Character " + str(char_x), str(char_y), str(char_w), str(char_h))
                
                charac = cropImage(char_x, char_y, char_w, char_h, word)
                #cv2.imshow('charac', charac)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                
                character = prediction(charac)
                character_list.append(character)
            word_list.append("".join(character_list))
    recognized_text = " ".join(word_list)
    return recognized_text"""

'def ocr(image):\n    \n    # detecting edges in the image\n    image_edges = cv2.Canny(image, 30, 150)\n    \n    # dilating image to detect individual lines\n    kernel_line = np.ones((10, 80), np.uint8)\n    dilated_line = cv2.dilate(image_edges, kernel_line, iterations=1)\n    \n    # finding contours of the line\n    im2, ctrs_line, hier = cv2.findContours(dilated_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n    sorted_ctrs_line = sorted(ctrs_line, key=lambda ctr: cv2.boundingRect(ctr)[1])\n    \n    # list of words\n    word_list = []\n    \n    for i, ctr_line in enumerate(sorted_ctrs_line):\n        # getting coordinates of the line contour\n        line_x, line_y, line_w, line_h = cv2.boundingRect(ctr_line)\n        \n        # if condition for removing unnecessary contours\n        if line_w * line_h < 10000:\n            continue\n        \n        line = cropImage(line_x, line_y, line_w, line_h, image)\n        cv2.imshow(\'line\', line)\n        cv2.waitKey(0)

In [ ]:
# Pran nonfilled
nonfilledimage = cv2.imread('nonfilledaadhar.jpg',0)

# IELTS nonfilled
#nonfilledimage = cv2.imread('nonfilled_0.jpg',0)

nonfilledimage = cv2.threshold(nonfilledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
nonfilledtesseract = nonfilledimage

nonfilledimagedilated = nonfilledimage.copy()
nonfilledimagedilated = cv2.dilate(nonfilledimagedilated, np.ones((2, 2), np.uint8), iterations=4)
nonfilledimage = nonfilledimagedilated

# Pran filled
filledimage = cv2.imread('filledaadhar.jpg',0)

# IELTS filled
#filledimage = cv2.imread('filled_0.jpg',0)

filledimage = cv2.threshold(filledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
filledimagedilated = cv2.dilate(filledimage, np.ones((2, 2), np.uint8), iterations=4)

# PRAN
#coordinates = [[196,336,1656,119],[191,981,2137,94],[920,2999,620,119],[176,1298,977,128]]
# aadhar
coordinates = [[243,585,951,89],[237,679,2003,79],[242,962,927,84],[1750,1309,515,89]]

label_dictionary = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a',
                    11: 'b', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'l', 20: 'm',
                    21: 'n', 22: 'q', 23: 'r', 24: 't', 25: 'y', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'E',
                    31: 'F', 32: 'G', 33: 'H', 34: 'I', 35: 'J', 36: 'K', 37: 'L', 38: 'M', 39: 'N', 40: 'O',
                    41: 'P', 42: 'Q', 43: 'R', 44: 'S', 45: 'T', 46: 'U', 47: 'V', 48: 'W', 49: 'X', 50: 'Y',
                    51: 'Z'}

# pran tags
#tags = ['Acknowledge No','First Name','Phone No','DOB']
# aadhar tags
tags = ['Pre Enrolment ID','Full Name:','House No BIdg Apt','PINCODE']
               
#tags_index = 0

test = cv2.imread('vrkIj.png',0)
test = cv2.threshold(nonfilledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
print(ocr(test))

#getNonfilledCroppedSections(coordinates)
#traverseCroppedSections()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from pprint import pprint
from keras.preprocessing import image as im
#import image
image = cv2.imread('vrkIj.png')
c = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#pprint(gray, indent=2)

# binary
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
#cv2.imshow('second', thresh)
# cv2.waitKey(0)

# dilation
kernel = np.ones((10, 80), np.uint8)
# original values 5,100
img_dilation = cv2.dilate(thresh, kernel, iterations=1)
#cv2.imshow('dilated', img_dilation)
# cv2.waitKey(0)

# find contours
im2, ctrs, hier = cv2.findContours(
    img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# sort contours
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[1])
i = 1
cropped_images = []
for i, ctr in enumerate(sorted_ctrs):
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)
    dimensions = [x, y, w, h]
    # Getting ROI
    roi = c[y:y + h, x:x + w]
    cropped_images.append(dimensions)
    #cv2.imwrite('test' + str(i) + '.jpg', roi)
    i += 1
    # show ROI

    #cv2.imshow('segment no:'+str(i),roi)
    cv2.rectangle(image, (x, y), (x + w, y + h), (90, 0, 255), 2)
    # cv2.waitKey(0)

j = 1
for x, y, w, h in cropped_images:
    img = c[y:y + h, x:x + w]
    d = img.copy()
    #cv2.imwrite('test' + str(j) + '.jpg', img)
    j += 1
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow('orig', gray)
    # cv2.waitKey(0)
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    #cv2.imshow('orig', thresh)
    # cv2.waitKey(0)
    kernel = np.ones((10, 15), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    #cv2.imshow('orig', img_dilation)
    #cv2.waitKey(0)
    im2, ctrs, hier = cv2.findContours(
        img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    i = 1
    #cv2.destroyAllWindows()
    images = []
    for i, ctr in enumerate(sorted_ctrs):
        # Get bounding box
        x, y, w, h = cv2.boundingRect(ctr)
        # Getting ROI
        roi = d[y:y + h, x:x + w]

        # cv2.imshow('test', roi)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        cv2.imwrite('test' + str(i) + '.jpg', roi)
        new_image = cv2.imread('test' + str(i) + '.jpg')
        gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        kernel = np.ones((10, 1), np.uint8)
        img_dilation = cv2.dilate(thresh, kernel, iterations=1)
        #cv2.imshow('dilation', img_dilation)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        im2, ctrs, hier = cv2.findContours(
            img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        e = thresh.copy()
        sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        character_list = []
        for i, ctr in enumerate(sorted_ctrs):
            # Get bounding box
            x, y, w, h = cv2.boundingRect(ctr)
            #dimensions = [x, y, w, h]
            # Getting ROI
            roi = e[y:y + h, x:x + w]
            squared = makeSquare(roi)
            final = resize_to_pixel(28, squared)
            
            predict_img = im.img_to_array(final)
            predict_img = np.expand_dims(predict_img, axis = 0)
            result = model.predict(predict_img)
            
            number = 0
            
            
            for i in result[0]:
                if i > 0:
                    character_list.append(number)
                number += 1
            
            #cv2.imshow('test', roi)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
            # cropped_images.append(dimensions)
            #cv2.imwrite('test' + str(i) + '.jpg', roi)
            #i += 1
            # show ROI

            #cv2.imshow('segment no:'+str(i),roi)
            #cv2.rectangle(image, (x, y), (x + w, y + h), (90, 0, 255), 2)
        words = []
        for character in character_list:
            words.append(label_dictionary[character])
        print("".join(words))
        """for character in character_list:
            print("".join())
            print("%s" % label_dictionary[character] )"""
        # for x_new, y_new, w_new, h_new in roi
        # cv2.imwrite('test' + str(i) + '.jpg', roi)
        #images.append([x, y, w, h])
        i += 1

In [5]:
def ocr(image):
    
    # detecting edges in the image
    image_edges = cv2.Canny(image, 30, 150)
    
    # dilating image to detect individual lines
    kernel_line = np.ones((10, 80), np.uint8)
    dilated_line = cv2.dilate(image_edges, kernel_line, iterations=1)
    cv2.imshow('dilated_line', dilated_line)
    cv2.waitKey(0)
    
    # finding contours of the line
    im2, ctrs_line, hier = cv2.findContours(dilated_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs_line = sorted(ctrs_line, key=lambda ctr: cv2.boundingRect(ctr)[1])
    
    # list of words
    word_list = []
    
    for i, ctr_line in enumerate(sorted_ctrs_line):
        # getting coordinates of the line contour
        line_x, line_y, line_w, line_h = cv2.boundingRect(ctr_line)
        
        # if condition for removing unnecessary contours
        #         if line_w * line_h < 10000:
        #             continue
        
        line = cropImage(line_x, line_y, line_w, line_h, image)
        cv2.imshow('line', line)
        cv2.waitKey(0)
        
        # detecting edges in the image
        line_edges = cv2.Canny(line, 30, 150)
        
        # dilating line to detect individual words
        kernel_word = np.ones((10, 60), np.uint8)
        dilated_word = cv2.dilate(line_edges, kernel_word, iterations=1)
        
        # finding contours of the word
        im2, ctrs_word, hier = cv2.findContours(dilated_word.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        sorted_ctrs_word = sorted(ctrs_word, key=lambda ctr: cv2.boundingRect(ctr)[0])
        
        for i, ctr_word in enumerate(sorted_ctrs_word):
            # getting coordinates of the word contour
            word_x, word_y, word_w, word_h = cv2.boundingRect(ctr_word)
            
            # if condition for removing unnecessary contours
            #             if word_w * word_h < 2500:
            #                 continue
            
            #print("Word " + str(word_x), str(word_y), str(word_w), str(word_h))
            
            word = cropImage(word_x, word_y, word_w, word_h, line)
            cv2.imshow('word', word)
            cv2.waitKey(0)
            
            word = cv2.erode(word, np.ones((1, 1), np.uint8), iterations=1)
            
            # detecting edges in the image
            word_edges = cv2.Canny(word, 30, 150)
            
            # dilating word to detect individual characters
            kernel_char = np.ones((15, 3), np.uint8)
            dilated_char = cv2.dilate(word_edges, kernel_char, iterations=1)
            cv2.imshow('dilated_char', dilated_char)
            cv2.waitKey(0)
            
            # finding contours of the characters
            im2, ctrs_char, hier = cv2.findContours(dilated_char.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            sorted_ctrs_char = sorted(ctrs_char, key=lambda ctr: cv2.boundingRect(ctr)[0])
            
            # list of words
            character_list = []
            
            for i, ctr_char in enumerate(sorted_ctrs_char):
                # getting coordinates of the character contour
                char_x, char_y, char_w, char_h = cv2.boundingRect(ctr_char)
                
                # if condition for removing unnecessary contours
                #                 if char_w * char_h < 180:
                #                     continue
                    
                #print("Character " + str(char_x), str(char_y), str(char_w), str(char_h))
                
                charac = cropImage(char_x, char_y, char_w, char_h, word)
                #cv2.imshow('charac', charac)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                
                character = prediction(charac)
                character_list.append(character)
            word_list.append("".join(character_list))
    recognized_text = " ".join(word_list)
    return recognized_text

In [2]:
test = cv2.imread('abc.jpg',0)
test = cv2.threshold(test, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imshow('char resized', test)
cv2.waitKey(0)
cv2.destroyAllWindows()
#print(ocr(test))
ocr(test)

MELL0 W0RLD aUlCK BR0WN F0X JUMPS heLl0 W0Rld 9UiCK J br0Wn f0X jUmPS


In [1]:
import cv2
import numpy as np
import tensorflow as tf

from keras.models import load_model
tf.reset_default_graph()
model=load_model('finalbestmodel.hdf5')
from keras.preprocessing import image as im

label_dictionary = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a',
                    11: 'b', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'l', 20: 'm',
                    21: 'n', 22: 'q', 23: 'r', 24: 't', 25: 'y', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'E',
                    31: 'F', 32: 'G', 33: 'H', 34: 'I', 35: 'J', 36: 'K', 37: 'L', 38: 'M', 39: 'N', 40: 'O',
                    41: 'P', 42: 'Q', 43: 'R', 44: 'S', 45: 'T', 46: 'U', 47: 'V', 48: 'W', 49: 'X', 50: 'Y',
                    51: 'Z'}

def makeSquare(not_square):
    # Adds black pixels as padding
    
    BLACK = [0, 0, 0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square, (2 * width, 2 * height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        if (height > width):
            pad = (height - width)/2
            pad = int(pad)
            #doublesize = int(doublesize)
            doublesize_square = cv2.copyMakeBorder(doublesize, 0, 0, pad,\
                                                  pad, cv2.BORDER_CONSTANT, value = BLACK)
        else:
            pad = (width - height)/2
            pad = int(pad)
            doublesize_square = cv2.copyMakeBorder(doublesize, pad, pad, 0, 0,\
                                                  cv2.BORDER_CONSTANT, value = BLACK)
    doublesize_square_dim = doublesize_square.shape
    return doublesize_square

def resize_to_pixel(dimensions, image):
    
    buffer_pix = 4
    dimensions = dimensions - buffer_pix
    squared = image
    r = float(dimensions)/squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    widht_r = img_dim2[1]
    BLACK = [0, 0, 0]
    if (height_r > widht_r):
        resized = cv2.copyMakeBorder(resized, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value = BLACK)
    if (height_r < widht_r):
        resized = cv2.copyMakeBorder(resized, 1, 0, 0, 0, cv2.BORDER_CONSTANT, value = BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized, p, p, p, p, cv2.BORDER_CONSTANT, value = BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    return ReSizedImg

def calculateClass(predictedarray):
    predictedclass = 0
    predictedclassindex = 0
    index = 0
    for classprediction in predictedarray:
        if classprediction > predictedclass:
            predictedclass = classprediction
            predictedclassindex = index
        index = index + 1
    return predictedclassindex

def prediction(char_image):
    squared = makeSquare(char_image)
    size28 = resize_to_pixel(28, squared)
    cv2.imshow('char resized', size28)
    cv2.waitKey(0)
    cv2.destroyAllWindows()     
    predict_img = im.img_to_array(size28)
    predict_img = np.expand_dims(predict_img, axis = 0)
    predictedarray = model.predict(predict_img)
    index = calculateClass(predictedarray[0])
    
    predicted_class = label_dictionary[index]
    return predicted_class

def cropImage(x, y, w, h, image):
    # Crop image here
    croppedImage = image[y:y + h,x:x + w]
    return croppedImage

def ocr(image):
    kernel_line = np.ones((10, 80), np.uint8)
    dilated_line = cv2.dilate(image, kernel_line, iterations=1)
    cv2.imshow('dilated_line', dilated_line)
    cv2.waitKey(0)
    
    im2, ctrs_line, hier = cv2.findContours(dilated_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs_line = sorted(ctrs_line, key=lambda ctr: cv2.boundingRect(ctr)[1])
    
    word_list = []
    
    for i, ctr_line in enumerate(sorted_ctrs_line):
        line_x, line_y, line_w, line_h = cv2.boundingRect(ctr_line)
        line = cropImage(line_x, line_y, line_w, line_h, image)
        
        kernel_word = np.ones((10, 40), np.uint8)
        dilated_word = cv2.dilate(line, kernel_word, iterations=1)
        cv2.imshow('dilated_word', dilated_word)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        im2, ctrs_word, hier = cv2.findContours(dilated_word.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        sorted_ctrs_word = sorted(ctrs_word, key=lambda ctr: cv2.boundingRect(ctr)[0])
        
        for i, ctr_word in enumerate(sorted_ctrs_word):
            word_x, word_y, word_w, word_h = cv2.boundingRect(ctr_word)
            
            word = cropImage(word_x, word_y, word_w, word_h, line)
            cv2.imshow('word', word)
            cv2.waitKey(0)
            
            word = cv2.erode(word, np.ones((1, 1), np.uint8), iterations=1)
            
            word_edges = cv2.Canny(word, 30, 150)
            
            kernel_char = np.ones((15, 2), np.uint8)
            dilated_char = cv2.dilate(word_edges, kernel_char, iterations=1)
            cv2.imshow('dilated_char', dilated_char)
            cv2.waitKey(0)
            
            im2, ctrs_char, hier = cv2.findContours(dilated_char.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            sorted_ctrs_char = sorted(ctrs_char, key=lambda ctr: cv2.boundingRect(ctr)[0])
            
            character_list = []
            
            for i, ctr_char in enumerate(sorted_ctrs_char):
                char_x, char_y, char_w, char_h = cv2.boundingRect(ctr_char)
                charac = cropImage(char_x, char_y, char_w, char_h, word)
                cv2.imshow('characsadsadasdasd', charac)
                cv2.waitKey(0)
                
                character = prediction(charac)
                character_list.append(character)
            correctedword = "".join(character_list)
            word_list.append(correctedword)
    recognized_text = " ".join(word_list)
    print(" ".join(word_list))
    #print(recognized_text)
        
    cv2.destroyAllWindows()

C:\Users\Praneet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
import re
from collections import Counter

def words(text):
	return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('./data/all_vocal_books.txt', encoding='utf-8').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = '0123456789abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
